<a href="https://colab.research.google.com/github/yuliiabosher/Cyber_Resilience_Course/blob/main/Testing_lambda_function_locally_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Testing the lambda function code locally

#### Installing the boto3 module

In [8]:
!pip install boto3

#### Importing the required modules

In [9]:
import pandas as pd
import io
import os
import json
import csv
import boto3
from IPython.display import clear_output

#### Saving an access key and a secret key in an environment variable
---

In [10]:
def set_environment_variable_values():
  ACCESS_KEY = input("Please enter the AWS access key: ")
  SECRET_ACCESS_KEY = input("Please enter the AWS secret access key: ")
  BUCKET_NAME = input("Please enter the name of the bucket in S3: ")
  os.environ['ACCESS_KEY'] = ACCESS_KEY
  os.environ['SECRET_ACCESS_KEY'] = SECRET_ACCESS_KEY
  os.environ['BUCKET_NAME'] = BUCKET_NAME
  clear_output()
  return None

set_environment_variable_values()

#### Creating a connection to the S3 bucket
---

In [23]:
def get_S3_client():
	resource = boto3.client(
     "s3",
		aws_access_key_id = os.environ.get('ACCESS_KEY'),
		aws_secret_access_key = os.environ.get('SECRET_ACCESS_KEY')
	)
	return resource

s3_client = get_S3_client()

#### Defining get_file(filename) function to test a GET request locally

---

In [24]:
def get_file(filename):
  # get the file from the bucket
  file_object = s3_client.get_object(Bucket=os.environ.get('BUCKET_NAME'), Key=filename)
  # convert the file object to a text-based csv file then read the file contents into a table using the pandas read_csv function
  data_file = io.BytesIO(file_object['Body'].read())
  data = pd.read_csv(data_file)
  return data

#### Defining save_a_copy(filedata, filename) function to test a POST request locally

---

In [25]:
def save_a_copy(filedata, filename):
  new_data = filedata.copy()
  file_object =  io.StringIO()
  pd.DataFrame(new_data).to_csv(file_object, index=False)
  response = s3_client.put_object(Bucket=os.environ.get('BUCKET_NAME'), Body=file_object.getvalue(), Key=filename)
  return response

#### Defining the lambda_handler(event, context) function from AWS Lambda with some minor edits
---

In [33]:
def lambda_handler(event, context):
    try:
        filename = "schools_list.csv"
        client = boto3.client('s3')
        if event["httpMethod"] == "POST":
            if "body" in event.keys():
                request = event["body"]
                if type(request) is not dict:
                    request = json.loads(request)
                if request is not None and "data" in request.keys():
                    '''if there is no data return a message and an empty list '''
                    data = request["data"]
                    if len(data) == 0:
                        message, return_data = "Please enter a valid data", []
                        statuscode = 404
                    else:
                        # message, return_data = add_school_data_to_bucket(client, filename, data)
                        return_data = save_a_copy(data,'schools_list.csv')
                        message="Success"
                        ''' the two lines above are different from the original Lambda function
                        to allow for them to be tested locally
                        the line that is hashed out is the original Lambda code '''
                        statuscode = 200
                else:
                    message, return_data = "Error in the POST request occured", []
                    statuscode = 404
        elif event["httpMethod"] == "GET":
            # message, return_data = show_schools_data_in_bucket(client, filename)
            return_data = get_file('schools_list.csv')
            message="Success"
            ''' the two lines above are different from the original Lambda function
            to allow for them to be tested locally
            the line that is hashed out is the original Lambda code '''
            statuscode = 200
        else:
            message, return_data = "Error occured", []
            statuscode = 404
        return {'statusCode': statuscode,
                'headers': {'Content-Type': 'application/json',
                            'Access-Control-Allow-Headers': 'Content-Type,X-Api-Key',
                            'Access-Control-Allow-Methods': 'POST',
                            'Access-Control-Allow-Origin': '*'},
                #'body': json.dumps({"message": message, "data": return_data})
                'body': {"message": message, "data": return_data}
                }
    except:
        return {'statusCode': 404,
                'headers': {'Content-Type': 'application/json',
                            'Access-Control-Allow-Headers': 'Content-Type,X-Api-Key',
                            'Access-Control-Allow-Methods': 'POST',
                            'Access-Control-Allow-Origin': '*'},
                # 'body': json.dumps({"message": "There was an error", "data": []})
                'body': {"message": "There was an error", "data": []}
                }
''' the line above is different from the original Lambda function
to allow for it to be tested locally
the line that is hashed out if the original Lambda code '''

' the line above is different from the original Lambda function \nto allow for it to be tested locally\nthe line that is hashed out if the original Lambda code '

#### Testing the lambda_handler(event, context) function using POST method
---
When POST method is used in the test, we check if the lambda_handler(event, context) function works locally using the save_a_copy(filedata, filename) function.

Expected output:

{'statusCode': 200,
 'headers': {'Content-Type': 'application/json',
  'Access-Control-Allow-Headers': 'Content-Type,X-Api-Key',
  'Access-Control-Allow-Methods': 'POST',
  'Access-Control-Allow-Origin': '*'},
 'body': {'message': 'Success',
  'data': {'ResponseMetadata': {'RequestId': '6R324YF334ANFT78',
    'HostId': 'pO1vh3CJyVGJk4wbUzaa59JOdccPBZPm8hcrHsf8D1orIN6uE05LSikpzVHluKp291WZnT0UDFo=',
    'HTTPStatusCode': 200,
    'HTTPHeaders': {'x-amz-id-2': 'pO1vh3CJyVGJk4wbUzaa59JOdccPBZPm8hcrHsf8D1orIN6uE05LSikpzVHluKp291WZnT0UDFo=',
     'x-amz-request-id': '6R324YF334ANFT78',
     'date': 'Fri, 15 Mar 2024 20:53:45 GMT',
     'x-amz-server-side-encryption': 'AES256',
     'etag': '"ddb9933f885c2a08fb1a37b457cb3b00"',
     'server': 'AmazonS3',
     'content-length': '0'},
    'RetryAttempts': 0},
   'ETag': '"ddb9933f885c2a08fb1a37b457cb3b00"',
   'ServerSideEncryption': 'AES256'}}}

In [34]:
event1 = {
    "httpMethod": "POST",
    "body": {
    "data": [
        [
            "Bannockburn Primary School & Nursery",
            51.4869172,
            0.1015561
        ],
        [
            "St Margaret Clitherow Primary School",
            51.50103289999999,
            0.1132992
        ]
    ]
}
}

lambda_handler(event1, None)

{'statusCode': 200,
 'headers': {'Content-Type': 'application/json',
  'Access-Control-Allow-Headers': 'Content-Type,X-Api-Key',
  'Access-Control-Allow-Methods': 'POST',
  'Access-Control-Allow-Origin': '*'},
 'body': {'message': 'Success',
  'data': {'ResponseMetadata': {'RequestId': '6R324YF334ANFT78',
    'HostId': 'pO1vh3CJyVGJk4wbUzaa59JOdccPBZPm8hcrHsf8D1orIN6uE05LSikpzVHluKp291WZnT0UDFo=',
    'HTTPStatusCode': 200,
    'HTTPHeaders': {'x-amz-id-2': 'pO1vh3CJyVGJk4wbUzaa59JOdccPBZPm8hcrHsf8D1orIN6uE05LSikpzVHluKp291WZnT0UDFo=',
     'x-amz-request-id': '6R324YF334ANFT78',
     'date': 'Fri, 15 Mar 2024 20:53:45 GMT',
     'x-amz-server-side-encryption': 'AES256',
     'etag': '"ddb9933f885c2a08fb1a37b457cb3b00"',
     'server': 'AmazonS3',
     'content-length': '0'},
    'RetryAttempts': 0},
   'ETag': '"ddb9933f885c2a08fb1a37b457cb3b00"',
   'ServerSideEncryption': 'AES256'}}}

#### Testing the lambda_handler(event, context) function using GET method
---
When GET method is used in the test, we check if the lambda_handler(event,context) function works locally using the get_file(filename) function.

Expected output:

{'statusCode': 200,
 'headers': {'Content-Type': 'application/json',
  'Access-Control-Allow-Headers': 'Content-Type,X-Api-Key',
  'Access-Control-Allow-Methods': 'POST',
  'Access-Control-Allow-Origin': '*'},
 'body': {'message': 'Success',
  'data':                                       0          1         2
  0  Bannockburn Primary School & Nursery  51.486917  0.101556
  1  St Margaret Clitherow Primary School  51.501033  0.113299}}

In [35]:
event2 = {
    "httpMethod": "GET"
}

lambda_handler(event2, None)

{'statusCode': 200,
 'headers': {'Content-Type': 'application/json',
  'Access-Control-Allow-Headers': 'Content-Type,X-Api-Key',
  'Access-Control-Allow-Methods': 'POST',
  'Access-Control-Allow-Origin': '*'},
 'body': {'message': 'Success',
  'data':                                       0          1         2
  0  Bannockburn Primary School & Nursery  51.486917  0.101556
  1  St Margaret Clitherow Primary School  51.501033  0.113299}}